In [1]:
import pandas as pd
import numpy as np
from rsome import ro, grb_solver as grb

# Data Ingestion

The datasets we used consist of data for 15 aircraft and 3 runways.

## Separation Times

In [2]:
separation_time = pd.read_csv("separation_time.csv", header=None)
separation_time.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,99999,1,3,3,3,3,3,3,3,3,1,1,3,3,1
1,1,99999,3,3,3,3,3,3,3,3,1,1,3,3,1
2,3,3,99999,2,2,2,2,2,2,2,3,3,2,2,3
3,3,3,2,99999,2,2,2,2,2,2,3,3,2,2,3
4,3,3,2,2,99999,2,2,2,2,2,3,3,2,2,3


In [3]:
separation_time = separation_time.values

## Aircraft

In [4]:
aircraft_df = pd.read_csv("aircraft.csv")
aircraft_df

,Aircraft ID,Earliest Landing Slot,Target Landing Slot,Latest Landing Slot,Earliness Penalty,Lateness Penalty,Size
0,0,3,4,12,10,30,3
1,1,4,6,15,10,30,3
2,2,1,2,11,10,30,2
3,3,2,2,11,10,30,2
4,4,3,3,11,10,30,2
5,5,3,5,12,10,30,2
6,6,3,3,12,10,30,1
7,7,5,9,11,10,30,1
8,8,6,10,12,10,30,1
9,9,3,4,13,10,30,1


In [5]:
earliest_landing = aircraft_df["Earliest Landing Slot"].values
target_landing = aircraft_df["Target Landing Slot"].values
latest_landing = aircraft_df["Latest Landing Slot"].values
earliness_penalty = aircraft_df["Earliness Penalty"].values
lateness_penalty = aircraft_df["Lateness Penalty"].values
aircraft_size = aircraft_df["Size"].values

## Runway

In [6]:
runway = pd.read_csv("runway.csv")
runway.head()

,Runway,Max Size
0,1,3
1,2,1
2,3,2


In [7]:
runway_max_size = runway["Max Size"].values

## Weather

In [8]:
good_weather = pd.read_csv("good_weather.csv")
good_weather.head()

,Good Weather
0,2
1,3
2,5
3,6
4,7


In [9]:
good_weather = good_weather["Good Weather"].values

# Model

In [10]:
def get_landing_schedule(
    separation_time,
    earliest_landing,
    target_landing,
    latest_landing,
    earliness_penalty,
    lateness_penalty,
    aircraft_size,
    runway_max_size,
    good_weather
):
    # Derivation of additional parameters
    # ===================================
    num_aircraft = len(earliest_landing)
    num_runways = len(runway_max_size)
    max_size_diff = aircraft_size.max() - runway_max_size.min()  # Max difference between runway size and aircraft size
    num_good_weather_slots = len(good_weather)
    M = 100
    
    # Initialisation
    # ==============
    model = ro.Model("Aircraft Landing Optimisation")
    
    # Decision variables
    # ==================
    # Scheduled landing time slot for each aircraft
    scheduled_landing = model.dvar(num_aircraft)

    # How early each aircraft is scheduled to land, with respect to the target landing time
    earliness = model.dvar(num_aircraft)
    
    # How late each aircraft is scheduled to land, with respect to the target landing time
    lateness = model.dvar(num_aircraft)
    
    # Whether aircraft i is scheduled to land in good weather slot g 
    i_lands_in_g = model.dvar((num_aircraft, num_good_weather_slots), 'B')
    
    # Whether aircraft i is scheduled to land before aircraft j
    i_lands_before_j = model.dvar((num_aircraft, num_aircraft), "B")
    
    # Whether aircraft i is scheduled to land on the same runway as aircraft j
    i_lands_on_same_runway_j = model.dvar((num_aircraft, num_aircraft), "B")
    
    # One-hot matrix of allocating runways to each aircraft
    runway_allocation = model.dvar((num_aircraft, num_runways), "B")  
    
    # Objective function
    # ==================
    model.min(sum(earliness_penalty*earliness) + sum(lateness_penalty*lateness))
    
    # Constraints
    # ===========
    # Since earliness, lateness > 0
    # If scheduled_landing - target_landing >= 0, then earliness >= 0 & lateness == 0
    # Else if scheduled_landing - target_landing <= 0, then earliness == 0 & lateness >= 0
    model.st(target_landing - scheduled_landing == earliness - lateness)
    
    # Each aircraft has to land between its respective latest and earliest time slot
    model.st(earliest_landing <= scheduled_landing)
    model.st(scheduled_landing <= latest_landing)
    
    for j in range(num_aircraft):
        for i in range(num_aircraft):
            if i != j:
                # The interval between 2 planes landing on the same runway must at least be equal to the separation time
                model.st(
                    scheduled_landing[j] - scheduled_landing[i] >= 
                    separation_time[i,j]*i_lands_on_same_runway_j[i,j] - M*i_lands_before_j[j,i]
                )

                # Every 2 planes must only either land before or after each other, not at the same time
                model.st(i_lands_before_j[i,j] + i_lands_before_j[j,i] == 1)

                # The link between i_lands_on_same_runway_j & runway_allocation
                for r in range(num_runways):
                    model.st(i_lands_on_same_runway_j[i,j] >= runway_allocation[i,r] + runway_allocation[j,r] - 1)
    
    # Aircraft size cannot be larger than max size of runway
    for i in range(num_aircraft):
        for r in range(num_runways):
            model.st(aircraft_size[i] - runway_max_size[r] <= (1 - runway_allocation[i,r])*max_size_diff)
            
    # Each aircraft i must only be allocated 1 runway
    for i in range(num_aircraft):
        model.st(sum(runway_allocation[i,r] for r in range(num_runways)) == 1)

    # Each aircraft i must be scheduled to land in a good weather time slot
    model.st(scheduled_landing[i] == good_weather @ i_lands_in_g[i] for i in range(num_aircraft))
    
    # Each aircraft i must only land on 1 good weather time slot
    model.st(i_lands_in_g[i].sum() == 1 for i in range(num_aircraft))
    
    # Non-negative constraints
    model.st(
        scheduled_landing >= 0,
        earliness >= 0,
        lateness >= 0
    )
    
    # Getting model information
    # =====================
    print(model.do_math())
    
    # Solving model
    # =============
    model.solve(grb)
    
    # Output generation
    # ================
    return scheduled_landing.get(), runway_allocation.get(), earliness.get(), lateness.get(), model.get()

In [25]:
scheduled_landing, runway_allocation, earliness, lateness, total_cost = get_landing_schedule(separation_time, 
                                                                                             earliest_landing,
                                                                                             target_landing,
                                                                                             latest_landing,
                                                                                             earliness_penalty,
                                                                                             lateness_penalty,
                                                                                             aircraft_size,
                                                                                             runway_max_size,
                                                                                             good_weather)

Conic program object:
Number of variables:           706
Continuous/binaries/integers:  46/660/0
---------------------------------------------
Number of linear constraints:  1156
Inequalities/equalities:       886/270
Number of coefficients:        3661
---------------------------------------------
Number of SOC constraints:     0
---------------------------------------------
Number of ExpCone constraints: 0

Being solved by Gurobi...
Solution status: 2
Running time: 5.8420s


# Output

In [26]:
# Generate output data frame
output_df = pd.DataFrame({
    "Aircraft ID": aircraft_df["Aircraft ID"].values,
    "Aircraft Size": aircraft_size,
    "Runway": np.argmax(runway_allocation, axis=1) + 1,
    "Scheduled Landing": scheduled_landing,
    "Target Landing": target_landing,
    "Earliest Landing": earliest_landing,
    "Latest Landing": latest_landing,
    "Earliness": earliness,
    "Lateness": lateness
})

print("Total cost:", total_cost)
print("Good weather:", good_weather)
print("All scheduled landings are in good weather:", all([i in good_weather for i in scheduled_landing]))

# Creating a new column for runway max size and shifting it to the 4th column
output_df["Runway Max Size"] = output_df['Runway'].apply(lambda x: runway_max_size[x-1])
temp = output_df["Runway Max Size"]
output_df.drop(labels=["Runway Max Size"], axis=1, inplace=True)
output_df.insert(3, "Runway Max Size", temp)
output_df

Total cost: 810.0
Good weather: [ 2  3  5  6  7 10 12 14 15 16 17]
All scheduled landings are in good weather: True


,Aircraft ID,Aircraft Size,Runway,Runway Max Size,Scheduled Landing,Target Landing,Earliest Landing,Latest Landing,Earliness,Lateness
0,0,3,1,3,5.0,4,3,12,0.0,1.0
1,1,3,1,3,6.0,6,4,15,0.0,0.0
2,2,2,3,2,2.0,2,1,11,0.0,0.0
3,3,2,1,3,2.0,2,2,11,0.0,0.0
4,4,2,3,2,10.0,3,3,11,0.0,7.0
5,5,2,3,2,7.0,5,3,12,0.0,2.0
6,6,1,2,1,3.0,3,3,12,0.0,0.0
7,7,1,2,1,7.0,9,5,11,2.0,0.0
8,8,1,1,3,12.0,10,6,12,0.0,2.0
9,9,1,2,1,5.0,4,3,13,0.0,1.0


The output above shows the allocated runways and scheduled landing slots for each aircraft. As observed, all aircraft are scheduled to land in good weather. No aircraft is allocated to a runway that is smaller than it. All scheduled landings also fall between the respective earliest and latest landings. The difference between the scheduled landing and target landing for each aircraft is indicated by the earliness and lateness columns.

# Re-running with Different Data

Let's try to change some input data. Specifically, let's modify some earliest and target landing times for some of the aircraft. The values modified are listed as follows:
* Aircraft 5
    * Target landing changed from 5 to 3.
* Aircraft 7
    * Earliest landing changed from 5 to 3.
    * Target landing changed from 9 to 3.
* Aircraft 8
    * Earliest landing changed from 6 to 3.
    * Target landing changed from 10 to 3.
* Aircraft 10
    * Target landing changed from 9 to 7.
* Aircraft 11
    * Target landing changed from 8 to 7.

The modified aircraft data is found in a file called `aircraft_v2.csv`.

In [27]:
aircraft_df = pd.read_csv("aircraft_v2.csv")

earliest_landing = aircraft_df["Earliest Landing Slot"].values
target_landing = aircraft_df["Target Landing Slot"].values
latest_landing = aircraft_df["Latest Landing Slot"].values
earliness_penalty = aircraft_df["Earliness Penalty"].values
lateness_penalty = aircraft_df["Lateness Penalty"].values
aircraft_size = aircraft_df["Size"].values

In [31]:
aircraft_df

,Aircraft ID,Earliest Landing Slot,Target Landing Slot,Latest Landing Slot,Earliness Penalty,Lateness Penalty,Size
0,0,3,4,12,10,30,3
1,1,4,6,15,10,30,3
2,2,1,2,11,10,30,2
3,3,2,2,11,10,30,2
4,4,3,3,11,10,30,2
5,5,3,3,12,10,30,2
6,6,3,3,12,10,30,1
7,7,3,3,11,10,30,1
8,8,3,3,12,10,30,1
9,9,3,4,13,10,30,1


In [28]:
scheduled_landing, runway_allocation, earliness, lateness, total_cost = get_landing_schedule(separation_time, 
                                                                                             earliest_landing,
                                                                                             target_landing,
                                                                                             latest_landing,
                                                                                             earliness_penalty,
                                                                                             lateness_penalty,
                                                                                             aircraft_size,
                                                                                             runway_max_size,
                                                                                             good_weather)

Conic program object:
Number of variables:           706
Continuous/binaries/integers:  46/660/0
---------------------------------------------
Number of linear constraints:  1156
Inequalities/equalities:       886/270
Number of coefficients:        3661
---------------------------------------------
Number of SOC constraints:     0
---------------------------------------------
Number of ExpCone constraints: 0

Being solved by Gurobi...
Solution status: 2
Running time: 375.4950s


As expected, the model arrives at a solution much more quickly when there is less input data.

In [32]:
output_df = pd.DataFrame({
    "Aircraft ID": aircraft_df["Aircraft ID"].values,
    "Aircraft Size": aircraft_size,
    "Runway": np.argmax(runway_allocation, axis=1) + 1,
    "Scheduled Landing": scheduled_landing,
    "Earliest Landing": earliest_landing,
    "Latest Landing": latest_landing,
    "Earliness": earliness,
    "Lateness": lateness
})

print("Total cost:", total_cost)
print("Good weather:", good_weather)
print("All scheduled landings are in good weather:", all([i in good_weather for i in scheduled_landing]))

output_df["Runway Max Size"] = output_df['Runway'].apply(lambda x: runway_max_size[x-1])
temp = output_df["Runway Max Size"]
output_df.drop(labels=["Runway Max Size"], axis=1, inplace=True)
output_df.insert(3, "Runway Max Size", temp)
output_df

Total cost: 1230.0
Good weather: [ 2  3  5  6  7 10 12 14 15 16 17]
All scheduled landings are in good weather: True


,Aircraft ID,Aircraft Size,Runway,Runway Max Size,Scheduled Landing,Earliest Landing,Latest Landing,Earliness,Lateness
0,0,3,1,3,5.0,3,12,0.0,1.0
1,1,3,1,3,6.0,4,15,0.0,0.0
2,2,2,1,3,2.0,1,11,0.0,0.0
3,3,2,3,2,2.0,2,11,0.0,0.0
4,4,2,3,2,10.0,3,11,0.0,7.0
5,5,2,1,3,10.0,3,12,0.0,7.0
6,6,1,2,1,3.0,3,12,0.0,0.0
7,7,1,2,1,7.0,3,11,0.0,4.0
8,8,1,3,2,7.0,3,12,0.0,4.0
9,9,1,2,1,5.0,3,13,0.0,1.0


# Visualisation of Results

In [15]:
import plotly.express as px

In [37]:
results_df = output_df[["Aircraft ID", "Runway", "Scheduled Landing", "Target Landing"]]

In [38]:
results_df = results_df.astype({"Aircraft ID": int, "Runway": str, "Scheduled Landing": int})
results_df = results_df.sort_values(by=["Runway"])

In [39]:
fig = px.scatter(results_df, x="Scheduled Landing", y="Runway", text="Aircraft ID",
                hover_data=list(results_df.columns), color="Runway", symbol="Runway")

fig.update_traces(textposition='top center')

fig.update_layout(
    title_text='Runway Allocation and Landing Schedule'
)

fig.show()